In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, concatenate

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, TimeSeriesSplit
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
root = '/kaggle/input/store-sales-time-series-forecasting/'

train_data = pd.read_csv(os.path.join(root, 'train.csv'), index_col=0)
test_data_orig = pd.read_csv(os.path.join(root, 'test.csv'), index_col=0)
oil_df = pd.read_csv(os.path.join(root, 'oil.csv'))
samp_subm = pd.read_csv(os.path.join(root, 'sample_submission.csv'))
holiday_event_df = pd.read_csv(os.path.join(root, 'holidays_events.csv'))
stores_df =  pd.read_csv(os.path.join(root, 'stores.csv'))
transactions_df = pd.read_csv(os.path.join(root, 'transactions.csv'))

In [3]:
# Merge
train_data = train_data.merge(stores_df, on = 'store_nbr') 
train_data = train_data.merge(oil_df, on ='date', how = 'left')
holiday_event_df = holiday_event_df.rename(columns = {'type': 'holiday'})
holiday_event_df['holiday'] = holiday_event_df['holiday'].apply(lambda x: 1 if x == 'Holiday' else 0)
holiday_event_df['holiday'].fillna(0, inplace=True)

train_data = train_data.merge(holiday_event_df, on = 'date', how = 'left')
test_data = test_data_orig.merge(holiday_event_df, on = 'date', how = 'left')

train_data['holiday'].fillna(0, inplace=True)
test_data['holiday'].fillna(0, inplace=True)

test_data['sales'] = 0.0
test_data_orig['sales'] = 0.0

In [4]:
train_data['holiday'].unique()

array([1., 0.])

In [5]:
train_data['holiday'].unique()

array([1., 0.])

In [6]:
req_columns = ['sales', 'onpromotion', 'holiday']
req_columns_agg = {'sales':'mean', 'onpromotion':'mean', 'holiday': 'max'}
# req_columns = {'sales':'mean'}
train_data = train_data.groupby(['date']).agg(req_columns_agg)
test_data = test_data.groupby(['date']).agg(req_columns_agg)

train_data = train_data[req_columns]
test_data = test_data[req_columns]

In [7]:
train_data

,sales,onpromotion,holiday
date,,,
2013-01-01,1.409438,0.000000,1.0
2013-01-02,278.390807,0.000000,0.0
2013-01-03,202.840197,0.000000,0.0
2013-01-04,198.911154,0.000000,0.0
2013-01-05,267.873244,0.000000,0.0
...,...,...,...
2017-08-11,463.733851,7.956790,0.0
2017-08-12,444.798280,4.664422,0.0
2017-08-13,485.768618,5.209315,0.0


In [8]:
test_data

,sales,onpromotion,holiday
date,,,
2017-08-16,0.0,17.137486,0.0
2017-08-17,0.0,4.211560,0.0
2017-08-18,0.0,7.731201,0.0
2017-08-19,0.0,4.833895,0.0
2017-08-20,0.0,5.338384,0.0
2017-08-21,0.0,4.545455,0.0
2017-08-22,0.0,5.777217,0.0
2017-08-23,0.0,9.259259,0.0
2017-08-24,0.0,4.639731,1.0


In [9]:
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        # label = data[i+seq_length]
        sequences.append(seq)
    # print(sequences)
    return np.array(sequences)
    
def get_ytrain(train_data, seq_length):
    # Extract the 'sales' column from the train data
    y_train = train_data['sales'].values.reshape(-1, 1)
    print(len(y_train))
    return y_train[seq_length:]

def get_xtrain_xtest(train_data, test_data, req_columns, seq_length=30):
    # Normalize the data
    # shift on promotion and holiday by one?
    # test_data_scaled = None
    # if add_cols:
    #     scaler = MinMaxScaler()
    #     scaler.fit_transform(train_data[add_cols])
    #     test_data_scaled = scaler.transform(test_data[add_cols])
    
    scaler = MinMaxScaler()
    train_data_scaled = scaler.fit_transform(train_data[req_columns])
    # test_data_scaled = scaler.transform(test_data[req_columns])
    
    train_data_scaled = create_sequences(train_data_scaled, seq_length)

    # return train_data_scaled, test_data_scaled, scaler
    return train_data_scaled, scaler

def get_next_seq(n, new_sale_val, train_data_scaled, test_data, scaler, seq_length):
    # create new instance in test_data
    new_instance = test_data.values[n]
    new_instance[0] = new_sale_val
    new_instance = np.array([new_instance])
    
    # transform the data to be added in xtrain
    print(f'New instance before transform: {new_instance}')
    new_instance_scaled =  scaler.transform(new_instance)
    print(f'New instance after transform: {new_instance_scaled}')
    
    # insert new value to x_train
    new_sequance = train_data_scaled[-1][1:]
    new_sequance = np.vstack([new_sequance, new_instance_scaled[0]])
    train_data_scaled = np.concatenate([train_data_scaled, new_sequance[np.newaxis, :, :]])
    # np.insert(train_data_scaled, len(train_data_scaled), new_instance_scaled[0], axis=0)
    
    # get new sequance. last seq_length items in x_train
    new_sequance = train_data_scaled[-1]
    new_sequance = np.array([new_sequance])    
    # print(new_sequance)
    
    return train_data_scaled, new_sequance

def get_predictions(model, test_data_orig, test_data, x_train, y_train, scaler, seq_length):
    new_sale_val = y_train[-1]
    for i in range(len(test_data)):
        x_train, new_sequance = get_next_seq(i, new_sale_val, x_train, test_data, scaler, seq_length)

        # predict
        sales_predicted = model.predict(new_sequance)
        print(sales_predicted)
        new_sale_val = sales_predicted[-1][0]
        date = test_data.index[i]
        print(f'Predict value for {date} : {new_sale_val}')
        
        # add predicted value to test_data
        test_data.iloc[i]['sales'] = new_sale_val
    
    return test_data

In [10]:
seq_length=30
x_train, scaler = get_xtrain_xtest(train_data, test_data, req_columns, seq_length=seq_length)
y_train = get_ytrain(train_data, seq_length)

1684


In [11]:
x_train

array([[[0.        , 0.        , 1.        ],
        [0.33793656, 0.        , 0.        ],
        [0.24575956, 0.        , 0.        ],
        ...,
        [0.19372443, 0.        , 0.        ],
        [0.17936612, 0.        , 0.        ],
        [0.19071257, 0.        , 0.        ]],

       [[0.33793656, 0.        , 0.        ],
        [0.24575956, 0.        , 0.        ],
        [0.24096585, 0.        , 0.        ],
        ...,
        [0.17936612, 0.        , 0.        ],
        [0.19071257, 0.        , 0.        ],
        [0.18399814, 0.        , 0.        ]],

       [[0.24575956, 0.        , 0.        ],
        [0.24096585, 0.        , 0.        ],
        [0.3251044 , 0.        , 0.        ],
        ...,
        [0.19071257, 0.        , 0.        ],
        [0.18399814, 0.        , 0.        ],
        [0.25119635, 0.        , 0.        ]],

       ...,

       [[0.52238289, 0.59785563, 0.        ],
        [0.66177174, 0.41245672, 0.        ],
        [0.75196007, 0

In [12]:
y_train

array([[152.21897755],
       [207.29632731],
       [291.182639  ],
       ...,
       [485.76861811],
       [427.00471722],
       [427.98088437]])

In [13]:
def validate_model(model, x_train, y_train, fit_args, n_splits=5):
    # Initialize lists to store evaluation metrics
    mse_scores = []
    mae_scores = []
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    i = 0
    # Perform k-fold cross-validation
    for train_index, val_index in tscv.split(x_train):
        # Split the data into training and validation sets
#         print(train_index)
        X_train, X_val = x_train[train_index], x_train[val_index]
        Y_train, Y_val = y_train[train_index], y_train[val_index]
        
        # Train the model
        model.fit(X_train, Y_train, **fit_args)

        # Make predictions on the validation set
        val_predictions = model.predict(X_val)

        # Evaluate the model
        mse = mean_squared_error(Y_val, val_predictions)
        print(f'mse of fold: {mse}')
        mae = mean_absolute_error(Y_val, val_predictions)
        print(f'mse of fold: {mae}')

        mse_scores.append(mse)
        mae_scores.append(mae)
        i += 1
    # Display the average performance metrics across folds
    print(f'Average Mean Squared Error: {np.mean(mse_scores)}')
    print(f'Average Mean Absolute Error: {np.mean(mae_scores)}')

In [14]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, x_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

fit_args = {'epochs': 500, 'batch_size': 32, 'verbose': 0}

In [16]:
validate_model(model, x_train, y_train, fit_args, n_splits=3)

13/13 [==============================] - 0s 5ms/step
mse of fold: 10151.479610389308
mse of fold: 74.80376098933944
13/13 [==============================] - 0s 4ms/step
mse of fold: 232567.3261969542
mse of fold: 219.53577485371065
13/13 [==============================] - 0s 4ms/step
mse of fold: 12483.630061638129
mse of fold: 82.80103075250499
Average Mean Squared Error: 85067.47862299388
Average Mean Absolute Error: 125.71352219851836


In [17]:
y_train.shape

(1654, 1)

In [18]:
# model.fit(X_train[:, :-1], X_train[:, -1], epochs=50, batch_size=32)

model.fit(x_train, y_train, **fit_args)

In [19]:
test_data = get_predictions(model, test_data_orig, test_data, x_train, y_train, scaler, seq_length)

New instance before transform: [[427.98088437  17.13748597   0.        ]]
New instance after transform: [[0.52044688 1.13692714 0.        ]]
1/1 [==============================] - 0s 19ms/step
[[99.69869]]
Predict value for 2017-08-16 : 99.69869232177734
New instance before transform: [[99.69869232  4.21156004  0.        ]]
New instance after transform: [[0.11991974 0.27940136 0.        ]]
1/1 [==============================] - 0s 19ms/step
[[133.11636]]
Predict value for 2017-08-17 : 133.11636352539062
New instance before transform: [[133.11636353   7.7312009    0.        ]]
New instance after transform: [[0.16069162 0.51289974 0.        ]]
1/1 [==============================] - 0s 19ms/step
[[97.852745]]
Predict value for 2017-08-18 : 97.85274505615234
New instance before transform: [[97.85274506  4.8338945   0.        ]]
New instance after transform: [[0.11766755 0.32068799 0.        ]]


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 19ms/step
[[52.00096]]
Predict value for 2017-08-19 : 52.00096130371094
New instance before transform: [[52.0009613   5.33838384  0.        ]]
New instance after transform: [[0.06172518 0.35415658 0.        ]]
1/1 [==============================] - 0s 20ms/step
[[199.41135]]
Predict value for 2017-08-20 : 199.41134643554688
New instance before transform: [[199.41134644   4.54545455   0.        ]]
New instance after transform: [[0.24157612 0.30155244 0.        ]]
1/1 [==============================] - 0s 19ms/step
[[158.80814]]
Predict value for 2017-08-21 : 158.80813598632812
New instance before transform: [[158.80813599   5.77721661   0.        ]]
New instance after transform: [[0.19203738 0.38326942 0.        ]]
1/1 [==============================] - 0s 17ms/step
[[164.1269]]
Predict value for 2017-08-22 : 164.1269073486328
New instance before transform: [[164.12690735   9.25925926   0.        ]]
New instance after transform: [[0.19852665 0.6

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 18ms/step
[[93.20022]]
Predict value for 2017-08-23 : 93.2002182006836
New instance before transform: [[93.2002182   4.63973064  1.        ]]
New instance after transform: [[0.11199115 0.30780686 1.        ]]
1/1 [==============================] - 0s 18ms/step
[[80.421684]]
Predict value for 2017-08-24 : 80.42168426513672
New instance before transform: [[80.42168427  7.85129068  0.        ]]
New instance after transform: [[0.09640044 0.52086668 0.        ]]
1/1 [==============================] - 0s 19ms/step
[[60.480553]]
Predict value for 2017-08-25 : 60.480552673339844
New instance before transform: [[60.48055267  5.08473625  0.        ]]
New instance after transform: [[0.07207087 0.33732921 0.        ]]
1/1 [==============================] - 0s 19ms/step
[[658.3584]]
Predict value for 2017-08-26 : 658.3583984375
New instance before transform: [[658.35839844   5.77833895   0.        ]]
New instance after transform: [[0.80152349 0.38334388 0. 

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 18ms/step
[[184.1184]]
Predict value for 2017-08-27 : 184.11839294433594
New instance before transform: [[184.11839294   4.7637486    0.        ]]
New instance after transform: [[0.22291765 0.3160344  0.        ]]
1/1 [==============================] - 0s 19ms/step
[[181.5577]]
Predict value for 2017-08-28 : 181.5576934814453
New instance before transform: [[181.55769348   6.10325477   0.        ]]
New instance after transform: [[0.21979342 0.4048993  0.        ]]
1/1 [==============================] - 0s 20ms/step
[[178.44046]]
Predict value for 2017-08-29 : 178.44046020507812
New instance before transform: [[178.44046021  12.06004489   0.        ]]
New instance after transform: [[0.21599018 0.8000819  0.        ]]
1/1 [==============================] - 0s 20ms/step
[[754.0537]]
Predict value for 2017-08-30 : 754.0537109375
New instance before transform: [[754.05371094   6.3305275    0.        ]]
New instance after transform: [[0.91827844 0.41

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 19ms/step
[[204.00221]]
Predict value for 2017-08-31 : 204.00221252441406


In [20]:
submission_df = test_data_orig.reset_index()
submission_df = submission_df.drop(columns=['sales'])
submission_df = pd.merge(submission_df, test_data, on='date', how='left')
submission_df

,id,date,store_nbr,family,onpromotion_x,sales,onpromotion_y,holiday
0,3000888,2017-08-16,1,AUTOMOTIVE,0,99.698692,17.137486,0.0
1,3000889,2017-08-16,1,BABY CARE,0,99.698692,17.137486,0.0
2,3000890,2017-08-16,1,BEAUTY,2,99.698692,17.137486,0.0
3,3000891,2017-08-16,1,BEVERAGES,20,99.698692,17.137486,0.0
4,3000892,2017-08-16,1,BOOKS,0,99.698692,17.137486,0.0
...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,204.002213,6.330527,0.0
28508,3029396,2017-08-31,9,PREPARED FOODS,0,204.002213,6.330527,0.0
28509,3029397,2017-08-31,9,PRODUCE,1,204.002213,6.330527,0.0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,204.002213,6.330527,0.0


In [21]:
submission_df = submission_df[['id', 'sales']]
submission_df

,id,sales
0,3000888,99.698692
1,3000889,99.698692
2,3000890,99.698692
3,3000891,99.698692
4,3000892,99.698692
...,...,...
28507,3029395,204.002213
28508,3029396,204.002213
28509,3029397,204.002213
28510,3029398,204.002213


In [22]:
submission_df.to_csv('submission.csv', index=False)

In [23]:
samp_subm

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
...,...,...
28507,3029395,0.0
28508,3029396,0.0
28509,3029397,0.0
28510,3029398,0.0


In [24]:
df = pd.read_csv('/kaggle/working/submission.csv')
df

,id,sales
0,3000888,99.698692
1,3000889,99.698692
2,3000890,99.698692
3,3000891,99.698692
4,3000892,99.698692
...,...,...
28507,3029395,204.002213
28508,3029396,204.002213
28509,3029397,204.002213
28510,3029398,204.002213
